In [1]:
import pandas as pd
import numpy as np
import datetime
import calendar
#import datetime from datetime
import math
import re
import pint
from sklearn import linear_model
data = pd.read_csv('fifa21_train.csv')
data.shape

(11701, 101)

In [2]:
new_col = []
for col in data.columns:
    new_col.append(col.lower())
data.columns = new_col
data.columns
    

Index(['id', 'name', 'age', 'nationality', 'club', 'bp', 'position',
       'team & contract', 'height', 'weight',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova'],
      dtype='object', length=101)

In [3]:
data.isna().sum()

id              0
name            0
age             0
nationality     0
club           18
               ..
cb              0
rcb             0
rb              0
gk              0
ova             0
Length: 101, dtype: int64

In [4]:
numeric = data.select_dtypes(include = np.number)
numeric.head()

,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,...,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,ova
0,184383,26,1,258,54,47,43,70,44.0,286,...,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,...,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,...,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,...,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,...,15,1664,360,66,44,60,64,60,66,65


In [5]:
categorical = data.select_dtypes(include = object)
categorical.head()

,name,nationality,club,bp,position,team & contract,height,weight,foot,joined,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015",...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2


In [6]:
categorical.isna().sum()

name                   0
nationality            0
club                  18
bp                     0
position             288
team & contract        0
height                 0
weight                 0
foot                   0
joined                41
loan date end      11072
value                  0
wage                   0
release clause         0
contract               0
w/f                    0
sm                     0
a/w                   59
d/w                   59
ir                     0
hits                   0
ls                     0
st                     0
rs                     0
lw                     0
lf                     0
cf                     0
rf                     0
rw                     0
lam                    0
cam                    0
ram                    0
lm                     0
lcm                    0
cm                     0
rcm                    0
rm                     0
lwb                    0
ldm                    0
cdm                    0


In [7]:
categorical['club'].value_counts()

OGC Nice          35
Crystal Palace    35
Everton           35
Chelsea           33
Sporting CP       33
                  ..
GIF Sundsvall      1
FC Orenburg        1
Germany            1
LKS Lodz           1
Santo André        1
Name: club, Length: 885, dtype: int64

In [8]:
#FOR PLAYERS WHERE CLUBS IS NA, WE ARE CLASSIFYING THEM AS FREE AGENTS
categorical['club'] = categorical['club'].fillna('free agent')

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_16820/1969520170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical['club'] = categorical['club'].fillna('free agent')


In [9]:
categorical['club'].isna().sum()

0

In [10]:
#FILL THE POSITION NaN WITH MODE
#WE SHALL CONSTRUCT A MODEL TO PREDICT THE PLAYER LATER
mode = categorical['position'].mode()[0]
mode
#LINK:https://stackoverflow.com/questions/21082671/find-and-select-the-most-frequent-data-of-column-in-pandas-dataframe

'CB'

In [11]:
#REPLACE NaN IN POSITION COLUMN WITH MODE
categorical['position'] = categorical['position'].fillna(mode)

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_16820/2505189410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical['position'] = categorical['position'].fillna(mode)


In [12]:
#CHECKING FOR NaN IN POSITION COLUMN
categorical['position'].isna().sum()

0

In [13]:
#SET NAN IN JOINED DATE TO 1ST SEPTEMBER 2020
#REASON: FiFA 21 is released usually in the second week of September or October. They usually stop collecting data on 1st September.
#So players who change clubs after that may not be updated in the game data
#SET DATE TO THE 1st DAY OF SEPTEMBER
first_day = datetime.date(2020, 9, 1)
display(first_day)
categorical['joined'] = categorical['joined'].fillna(first_day.strftime('%b %d, %Y'))

datetime.date(2020, 9, 1)

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_16820/2129599596.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical['joined'] = categorical['joined'].fillna(first_day.strftime('%b %d, %Y'))


In [14]:
categorical['joined'].isna().sum()

0

In [15]:
#GETTING RATIO OF NaN TO ACTUAL VALUES IN THE 'Loan Date End' COLUMN TO LENGTH OF DF
display(categorical['loan date end'].isna().sum())
perc = (categorical['loan date end'].isna().sum()/data.shape[0]) * 100
perc

11072

94.62439107768567

In [16]:
#OVER 94% OF THE VALUES IN THE 'Loan Date End' ARE NaN. SO, WE ARE DROPPING IT
categorical = categorical.drop(['loan date end'], axis = 1)
categorical.columns

Index(['name', 'nationality', 'club', 'bp', 'position', 'team & contract',
       'height', 'weight', 'foot', 'joined', 'value', 'wage', 'release clause',
       'contract', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'hits', 'ls', 'st', 'rs',
       'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
       'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb', 'gk'],
      dtype='object')

In [17]:
# d = pd.Period('Dec 01, 2021',freq='M').end_time.date() 
# print(d)   

In [18]:
x = '80 + 2'
y = int(x.split('+')[0]) + int(x.split('+')[1])
y

82

In [19]:
numeric.isna().sum()

id                    0
age                   0
growth                0
attacking             0
crossing              0
finishing             0
heading accuracy      0
short passing         0
volleys              41
skill                 0
dribbling             0
curve                41
fk accuracy           0
long passing          0
ball control          0
movement              0
acceleration          0
sprint speed          0
agility              41
reactions             0
balance              41
power                 0
shot power            0
jumping              41
stamina               0
strength              0
long shots            0
mentality             0
aggression            0
interceptions         7
positioning           7
vision               41
penalties             0
composure           279
defending             0
marking               0
standing tackle       0
sliding tackle       41
goalkeeping           0
gk diving             0
gk handling           0
gk kicking      

In [20]:
#REPLACE THE NaN WITH THE AVERAGE OF ALL PLAYERS IN SIMILAR POSITION
data['position'].value_counts()

CB           1352
ST           1267
GK           1040
CDM CM        515
CM            449
             ... 
RB CM CAM       1
CF RM RW        1
LB LWB RB       1
LB CM LW        1
RW LM CAM       1
Name: position, Length: 543, dtype: int64

In [21]:
numeric['jumping'].isna().sum()

41

In [22]:
# GET A LIST OF ALL COLS WITH NaN VALUES IN THE NUMERIC DATAFRAME
NaN_col_list = []
for col in numeric.columns:
    if numeric[col].isna().sum() != 0:
        NaN_col_list.append(col)
NaN_col_list        

['volleys',
 'curve',
 'agility',
 'balance',
 'jumping',
 'interceptions',
 'positioning',
 'vision',
 'composure',
 'sliding tackle']

In [23]:
#REPLACE ALL NaN VALUES WITH THE MEAN
for colx in NaN_col_list:
    mean = np.mean(numeric[colx])
    mean = math.ceil(mean)
    #display(mean)
    numeric[colx] = numeric[colx].fillna(mean)

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_16820/252384581.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric[colx] = numeric[colx].fillna(mean)


In [24]:
numeric.isna().sum()

id                  0
age                 0
growth              0
attacking           0
crossing            0
finishing           0
heading accuracy    0
short passing       0
volleys             0
skill               0
dribbling           0
curve               0
fk accuracy         0
long passing        0
ball control        0
movement            0
acceleration        0
sprint speed        0
agility             0
reactions           0
balance             0
power               0
shot power          0
jumping             0
stamina             0
strength            0
long shots          0
mentality           0
aggression          0
interceptions       0
positioning         0
vision              0
penalties           0
composure           0
defending           0
marking             0
standing tackle     0
sliding tackle      0
goalkeeping         0
gk diving           0
gk handling         0
gk kicking          0
gk positioning      0
gk reflexes         0
total stats         0
base stats

In [25]:
#STANDARDISE ALL NUMERIC VALUES WITH THE MeanMaxScalar
from sklearn.preprocessing import MinMaxScaler
kimera = MinMaxScaler().fit(numeric)
x_normalized = kimera.transform(numeric)
print(x_normalized.shape)
x_normalized = pd.DataFrame(x_normalized,columns=numeric.columns)
x_normalized.head()

(11701, 53)


,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,...,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,ova
0,0.711626,0.370370,0.074074,0.556931,0.545455,0.478261,0.431818,0.720930,0.465116,0.569087,...,0.056180,0.604577,0.492366,0.614286,0.447368,0.558824,0.522388,0.506494,0.500000,0.472727
1,0.725758,0.518519,0.037037,0.821782,0.681818,0.826087,0.806818,0.697674,0.837209,0.777518,...,0.168539,0.781945,0.702290,0.814286,0.763158,0.632353,0.805970,0.272727,0.666667,0.709091
2,0.711811,0.629630,0.037037,0.750000,0.761364,0.793478,0.329545,0.813953,0.825581,0.892272,...,0.022472,0.759059,0.671756,0.771429,0.789474,0.779412,0.865672,0.194805,0.439394,0.763636
3,0.902364,0.222222,0.518519,0.517327,0.431818,0.423913,0.602273,0.627907,0.372093,0.505855,...,0.123596,0.506039,0.385496,0.442857,0.355263,0.426471,0.432836,0.584416,0.500000,0.381818
4,0.906235,0.259259,0.333333,0.534653,0.488636,0.369565,0.636364,0.697674,0.348837,0.555035,...,0.157303,0.593134,0.503817,0.571429,0.355263,0.514706,0.537313,0.623377,0.590909,0.490909


In [26]:
categorical.columns

Index(['name', 'nationality', 'club', 'bp', 'position', 'team & contract',
       'height', 'weight', 'foot', 'joined', 'value', 'wage', 'release clause',
       'contract', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'hits', 'ls', 'st', 'rs',
       'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
       'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb', 'gk'],
      dtype='object')

In [27]:
# GET ONLY RELEVANT COLUMNS FROM THE CATEGORICAL DATAFRAMES
# ALOT OF COLUMNS IN THE DATAFRAME ARE NOT RELEVANT TO THE PLAYER'S OVERALL RATING
# COLUMNS LIKE CLUB, NATIONALITY, TEAM & COUNTRY HAVE NO IMPACT ON THE PLAYER'S OVERALL RATINGS
categorical = categorical[['position','height','weight','foot']]

In [28]:
for coln in categorical.columns:
    display(categorical[coln].value_counts())

CB           1640
ST           1267
GK           1040
CDM CM        515
CM            449
             ... 
RB CM CAM       1
CF RM RW        1
LB LWB RB       1
LB CM LW        1
RW LM CAM       1
Name: position, Length: 543, dtype: int64

6'0"     1751
5'10"    1636
5'9"     1401
5'11"    1314
6'2"     1261
6'1"     1257
6'3"      659
5'8"      657
5'7"      614
6'4"      497
5'6"      209
6'5"      178
5'5"      129
6'6"       75
5'4"       19
6'7"       19
6'8"       10
5'3"        9
5'2"        3
5'1"        2
6'9"        1
Name: height, dtype: int64

154lbs    931
165lbs    871
176lbs    694
159lbs    651
172lbs    626
161lbs    605
163lbs    573
168lbs    555
170lbs    506
150lbs    453
157lbs    425
174lbs    406
181lbs    396
143lbs    338
183lbs    332
152lbs    324
179lbs    312
148lbs    295
185lbs    271
187lbs    262
146lbs    244
141lbs    207
190lbs    175
194lbs    159
139lbs    150
192lbs    125
198lbs    110
137lbs    106
132lbs    105
196lbs     91
134lbs     82
201lbs     56
203lbs     48
205lbs     33
209lbs     32
128lbs     29
207lbs     26
130lbs     19
126lbs     11
212lbs     11
214lbs     10
123lbs      9
216lbs      8
121lbs      7
220lbs      5
225lbs      4
218lbs      3
227lbs      2
223lbs      2
117lbs      1
236lbs      1
243lbs      1
229lbs      1
119lbs      1
234lbs      1
Name: weight, dtype: int64

Right    8828
Left     2873
Name: foot, dtype: int64

In [29]:
categorical['position'] = categorical['position'].apply(lambda x: x.split(' ')[0])
categorical['position']
display(categorical['position'].value_counts())

CB      2139
ST      1803
CM      1310
GK      1040
CDM      878
LB       839
RB       821
CAM      759
LM       687
RM       667
LW       295
RW       271
CF        87
RWB       53
LWB       48
ACB        1
IVST       1
IIRM       1
ND         1
Name: position, dtype: int64

In [30]:
#REMOVE ALL PLAYERS WHOSE POSITIONS DO NOT APPEAR AMONG THE PROVIDED POSITIONS
#REASON: They are few - 4 players in total

drop_list = ['ACB','IVST','IIRM','ND']
for item in drop_list:
    categorical = categorical.drop(categorical.loc[categorical['position']==item].index)
display(categorical['position'].value_counts())

CB     2139
ST     1803
CM     1310
GK     1040
CDM     878
LB      839
RB      821
CAM     759
LM      687
RM      667
LW      295
RW      271
CF       87
RWB      53
LWB      48
Name: position, dtype: int64

In [31]:
# THE COLUMNS NEED TO BE TREATED DIFFERENTLY. FOOT HAS TWO UNIQUE VALUES
# WE NEED TO CONVERT WEIGHT TO INT OR FLOAT BECAUSE A PLAYER'S WEIGHT IS A NUMBER
# CONVERT THE PLAYER'S HEIGHT TO METERS
# PLAYER ARE GIVEN VARIOUS POSITIONAL COMBINTATIONS. I WILL TAKE THE FIRST POSITION AS THE PLAYER'S POSITION

In [32]:
# CONVERT WEIGHT TO INTEGER BY REMOVING THE 'lbs' AT THE END AND CONVERT STRING INTO INT 

categorical['weight'] = categorical['weight'].apply(lambda x: int(x.replace('lbs', '')))
categorical['weight']

0        161
1        159
2        134
3        152
4        150
        ... 
11696    168
11697    174
11698    143
11699    179
11700    143
Name: weight, Length: 11697, dtype: int64

In [33]:
# CONVERT THE HEIGHT FROM STRING TO METERS
# TO BE DONE LATER. REPLACING THE FEET & INCHES SIGNS IS PROVING VERY CHALLENGING
# USE THE FIRST INTEGER
ureg = pint.UnitRegistry()
def replace_char(x):    
    x = re.sub('[^a-zA-Z0-9 \n\.]', '', x)
    if len(x) > 2:             
        y = int(x[0]) * ureg.feet
        z = int(x[-2]) * ureg.inch
        x = round(y.to(ureg.meter) + z.to(ureg.meter),2)
    
    elif len(x) == 2:
        y = int(x[0])* ureg.feet
        z = int(x[-1]) * ureg.inch
        x = round(y.to(ureg.meter) + z.to(ureg.meter),2)
    return x.magnitude
    #REMOVES THE meter AND RETURN THE MAGNITUDE ONLY

categorical['height'] = categorical['height'].apply(lambda x: replace_char(x))
categorical['height'].head()

0    1.75
1    1.83
2    1.63
3    1.55
4    1.55
Name: height, dtype: float64

In [34]:
print(categorical.columns)
categorical['foot'].value_counts()

Index(['position', 'height', 'weight', 'foot'], dtype='object')


Right    8826
Left     2871
Name: foot, dtype: int64

In [35]:
categorical['foot'].head()

0    Right
1    Right
2    Right
3    Right
4    Right
Name: foot, dtype: object

In [36]:
# #FOOT HAS ONLY TWO UNIQUE VALUES. STANDARDISE USING ONE ENCODER
# from sklearn.preprocessing import OneHotEncoder
# #X_cat = categorical['foot']
# encoder = OneHotEncoder().fit(categorical['foot'])
# print(encoder)

In [37]:
# FOOT IS REMOVED BECAUSE OF THE FOLLOWING ERRORS
# ValueError: Expected 2D array, got 1D array instead:
# array=['Right' 'Right' 'Right' ... 'Right' 'Right' 'Left'].
categorical = categorical.drop(['foot'], axis = 1)
categorical.columns

Index(['position', 'height', 'weight'], dtype='object')

In [38]:
# let's merge all this information together into a single dataset with all features, now numerical
X = pd.concat([numeric, categorical], axis=1)
X

,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,...,pac,sho,pas,dri,def,phy,ova,position,height,weight
0,184383,26,1,258,54,47,43,70,44.0,286,...,69,51,63,63,51,60,64,CM,1.75,161.0
1,188044,30,0,365,66,79,76,68,76.0,375,...,83,75,68,82,33,71,77,ST,1.83,159.0
2,184431,33,0,336,73,76,34,78,75.0,424,...,80,77,78,86,27,56,80,CAM,1.63,134.0
3,233796,22,13,242,44,42,58,62,36.0,259,...,57,44,54,57,57,60,59,CDM,1.55,152.0
4,234799,23,8,249,49,37,61,68,34.0,280,...,66,44,60,64,60,66,65,CDM,1.55,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,25,5,224,62,24,51,59,28.0,248,...,75,28,56,59,60,69,65,LB,1.85,168.0
11697,214680,27,4,84,12,14,13,29,16.0,96,...,72,67,66,72,40,70,70,GK,1.85,174.0
11698,221489,22,8,286,64,66,51,64,41.0,291,...,76,60,60,72,26,43,67,LM,1.68,143.0
11699,146717,26,7,254,64,66,51,73,46.0,284,...,72,63,70,72,57,66,68,CB,1.88,179.0
